In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms, datasets
import os

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/data/ant_bee'
transforms = transforms.Compose([transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# create the training and val dataloader
train_dataset = datasets.ImageFolder(os.path.join(path, 'train'), transform=transforms)
val_dataset = datasets.ImageFolder(os.path.join(path, 'val'), transform = transforms)

# Creating the batches
train_dataloader = DataLoader(train_dataset, batch_size = 16, drop_last = True, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = 16, drop_last = True, shuffle = True)

In [ ]:
print(f'Shape of the input images are {next(iter(train_dataloader))[0].shape}')
print(f'Shape of the input labels are {next(iter(train_dataloader))[1].shape}')

In [ ]:
# loading our model
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False


num_in_ftrs = model.fc.in_features # finding the number of input features
# model.fc = nn.Linear(num_in_ftrs, 2) # we have two classes
model.fc = nn.Sequential(nn.Linear(num_in_ftrs, 128), 
                         nn.ReLU(),
                         nn.Dropout(0.2),
                         nn.Linear(128, 16), 
                         nn.ReLU(),
                         nn.Dropout(0.2),
                         nn.Linear(16, 2), 
                         nn.Softmax())

In [ ]:
# Create the criterion 
criterion = nn.CrossEntropyLoss()

# Create the optimizer
optimizer = torch.optim.Adam(model.fc.parameters(), lr = 0.01)

# Selecting the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_loss_epoch = []
val_loss_epoch = []
val_accuracy = []

model.to(device)
criterion.to(device)

epochs = 30

for epoch in range(epochs):

  model.train()

  train_loss = 0
  for images, labels in train_dataloader:

    optimizer.zero_grad()

    images = images.to(device)
    labels = labels.to(device)

    logits = model(images)
    loss = criterion(logits, labels)

    print(logits)

    loss.backward()

    optimizer.step()

    train_loss+=loss


  model.eval()
  size = len(val_dataset)

  with torch.no_grad():
    val_loss = 0
    running_corrects = 0
    for images, labels in val_dataloader:
      
      images = images.to(device)
      labels = labels.to(device)

      output = model(images)
      loss = criterion(output, labels)

      val_loss += loss


      # Calculate the accuracy
      _, preds = torch.max(outputs, 1)
      running_corrects += torch.sum(preds == labels.data)

  print(f'Epoch no : {epoch}')
  print('_' * 30)
  print(f'Train loss for {epoch}/{epochs}: {train_loss}')
  print(f'Val loss for {epoch}/{epochs}: {val_loss}')
  print(f'Validation accuracy: {running_corrects/size}')


  train_loss_epoch.append(train_loss)
  val_loss_epoch.append(val_loss)
  val_accuracy.append(running_corrects)


In [ ]:
len(val_dataset)